In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize 

In [458]:
class RNN:
    def __init__ (self, numHidden, numInput, numOutput):
        self.numHidden = numHidden
        self.numInput = numInput
        self.U = np.random.randn(numHidden, numHidden) * 1e-1
        self.V = np.random.randn(numHidden, numInput) * 1e-1
        self.w = np.array(np.random.randn(numHidden) * 1e-1)
        self.h = np.zeros((numHidden,51))
        self.z = np.zeros((numHidden,50))
        self.yh = np.zeros(50)
        self.g = []
        
        # TODO: IMPLEMENT ME
    def dj_du(self,t,y):
        r = 0
        for i in range(t-1,-1,-1):
            if i==t-1:
                q = np.multiply((np.dot((self.yh[i]-y[i]),self.w)),self.g[i].T)
            else:
                q = np.multiply(np.dot(q,self.U),self.g[i].T)
            #print("q shape",q.shape)
            #print("h shape",self.h[:,i].shape)
            r += np.dot(q.reshape(6,1),self.h[:,i].reshape(1,6))
            #print("r shape",r.shape,r)
            assert r.shape == (6,6)
        return r

    def dj_dv(self,t,y,x):
        e = 0
        for i in range(t,-1,-1):
            #print("tao",i)
            #print("g"+str(i),self.g[i])
            if i==t:
                q = np.multiply((y[i]-self.yh[i])*self.w,self.g[i])
            else:
                q = np.multiply(np.dot(q,self.U),self.g[i])
            e += np.multiply(q.reshape(6,1),x[i])
            assert e.shape == (6,1) 
            #print("e",e)
        return e
    
    def dj_dw(self,t,y):
        a = np.dot((self.yh[t]-y[t]),(self.h[:,t]).reshape(6,1))
        assert a.shape == (6,1)
        #print(a.shape)
        #print(a)
        return a
    def backward (self,x,y,tp,alpha):
        du,dv,dw,loss = 0,0,0,0
        for i in range(tp):
            #print("time",i)
            threshold = 200
            self.forward(x)
            du = self.dj_du(i,y)
            dv = self.dj_dv(i,y,x)
            dw = self.dj_dw(i,y)
            if np.any(du > threshold):
                print(1)
                du = du / np.max(du) * threshold
            if np.any(du < -threshold):
                print(1)
                du = -du / np.min(du) * threshold
            if np.any(dv > threshold):
                print(2)
                dv = dv / np.max(dv) * threshold
            if np.any(dv < -threshold):
                print(2)
                dv = -dv / np.min(dv) * threshold
            self.U -= alpha * du
            self.V -= (alpha * dv).reshape(6,1)
            (self.w).reshape(1,6)
            #print(self.w)
            #print(self.w.shape)
            #print(dw)
            #print((alpha * dw).shape)
            #print((self.w - alpha*dw.T).shape)
            #print(self.w - alpha*dw.T)
            self.w = (self.w-alpha * dw.T)
            loss += 1/2*(self.yh[i]-y[i])**2
        return loss
    def forward (self, x):
        self.g = []
        for i in range(x.shape[0]):
            #print("i",i)
            self.z[:,i] = np.dot(self.U,self.h[:,i])+np.dot(self.V,np.array([x[i]]))
            self.h[:,i+1] = np.tanh(self.z[:,i])
            self.yh[i] = np.dot(self.w,self.h[:,i+1])
            self.g.append(1-(np.tanh(self.z[:,i])**2).T)
            #print("z"+str(i+1),self.z[:,i])
            #print("h"+str(i+1),self.h[:i+1])
            #print("g"+str(i+1),self.g[i])
            #print(self.h.shape)
        return self.yh

In [459]:
def generateData ():
    total_series_length = 50
    echo_step = 2  # 2-back task
    batch_size = 1
    x = np.random.choice(2, total_series_length, p=[0.5, 0.5])
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0
    y = list(y)
    return (x, y)

In [ ]:
if __name__ == "__main__":
    xs, ys = generateData()
    numHidden = 6
    numInput = 1
    numTimesteps = len(xs)
    rnn = RNN(numHidden, numInput, 1)
    # TODO: IMPLEMENT ME
    epochs = 150000
    alpha = 0.001
    loss = 0
    for i in range(epochs):
        loss = rnn.backward(xs,ys,numTimesteps,alpha)
        if i%500==0:
            print("epoch: %s"% (i+1))
            print(loss)
        if loss < 0.05:
            break

epoch: 1
12.139876334833954
epoch: 501
11.813084830017404
epoch: 1001
13.243522111655114
epoch: 1501
12.081514402010532
epoch: 2001
10.206290568435191
epoch: 2501
12.563796672855757
epoch: 3001
6.077323061572788
epoch: 3501
6.033148439506702
epoch: 4001
7.204361033737857
epoch: 4501
6.468394811275627
epoch: 5001
6.197988711308767
epoch: 5501
6.168465938072886
epoch: 6001
5.746512810025048
epoch: 6501
7.735907239080507
epoch: 7001
10.641023394967025
epoch: 7501
17.451809096394815
epoch: 8001
19.829010995933608
